In [17]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence, TaggedLineDocument
from gensim.models import Doc2Vec

# numpy
import numpy as np

# random
from random import shuffle

In [11]:
import nltk
import pandas as pd
from pandas import Series, DataFrame, Panel

#allcomments=pd.read_csv('C:\\Users\\Raven.Raven-PC\\Desktop\\hedgehog\\moneycontrolbis\\allcommentstest.csv')
#allcomments=pd.read_csv('C:\\Users\\Fitec\\Downloads\\allcommentstest.csv')
allcomments=pd.read_csv('C:\\Users\\Fitec\\Desktop\\yahoofinance\\NewData\\DataCommentsYahoo.csv')
justcomments=allcomments['Comments']


In [12]:
type(justcomments[1])

str

In [13]:
import re
from html.parser import HTMLParser
html_parser = HTMLParser()
#test="I luv my &lt;3 iphone &amp; you’re awsm apple. DisplayIsAwesome, sooo happppppy 🙂 http://www.apple.com"
#parsedtest=html_parser.unescape(test)
#print(parsedtest)
parsedcomments=[]
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

for i in justcomments:
    #j= i.decode("utf8").encode("ascii","ignore")
    htmlparsedcomment = html_parser.unescape(str(i))
    htmlparsedcomment=htmlparsedcomment.lower()
    htmlparsedcomment = re.sub(r'^https?:\/\/.*[\r\n]*', '', htmlparsedcomment, flags=re.MULTILINE)
    htmlparsedcomment = re.sub(r'[^\w\s]','',htmlparsedcomment)
    htmlparsedcomment=emoji_pattern.sub(r'', htmlparsedcomment)
    htmlparsedcomment=re.sub(r'[^\x00-\x7f]',r'', htmlparsedcomment) 
    htmlparsedcomment=htmlparsedcomment.replace('\n', '')
    #s = s.encode('ascii',errors='ignore')
    parsedcomments.append(htmlparsedcomment)
    #Cleaning checkpoint 1 - To be explored
print(parsedcomments[1:10])

C:\Users\Fitec\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.


['aether once again pushes the 3d printing sector sky high  incredible', 'almost sold my calls this morning good thing almost doesnt count', 'i just want ddd to get to 75 then i think ill take some profit', 'at drudgereport there is a good article about the 3d printer spacennmaybe the sector is getting hot again like years ago', 'its clear that the reseller surveys are telling a story of ddd winning out over ssys  this runup while certainly part shortsqueeze is based on something more  substantial  and since no deal has surfaced i dont think thats what is coming  a few more printers being sold leads to the increase eps at the materials marginsnnnothing about this runup is based on what has been happening more on what is improving', '6539 calls wu002fstrike at 20 been outstanding for a long time is shorty looking to scoop up 650000 and then drop them on the market', 'no risk no fun just went in will top up if it goes up to 25', 'the lack of participation in this rally by hpq suggests th

In [14]:
commentsfile=open('comments4.txt','w')
for item in parsedcomments:
    if item.rstrip():                 #this is only to remove blank lines if any in the text file 
        commentsfile.write("%s\n" % item)

    
        

In [23]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.cluster import KMeans
import time



# random
from random import shuffle

# classifier

class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

sources = {'comments4.txt' : 'DOCS'}
sentences = LabeledLineSentence(sources)


for dimsize in range(10,80,10):
    print(str(dimsize))
    tic = time.clock()

    model = Doc2Vec(min_count=1, window=10, size=dimsize, sample=1e-4, negative=5, workers=8)

    toc1= time.clock()
    timeformodel=toc1-tic
    print("timeformodel:"+str(timeformodel))
    model.build_vocab(sentences.to_array())
    toc2= time.clock()
    timeforbuildvocab=toc2-toc1
    print("timeforbuildvocab:"+str(timeforbuildvocab))
    for epoch in range(10):

        model.train(sentences.sentences_perm())
        toc3= time.clock()
        print("timefortraintillnow:"+str(toc3-toc2))

    model.save('doc2veccomplexitytest'+str(dimsize))
#model = Word2Vec.load(fname) - TO LOAD the MODEL
#w2v = dict(zip(model.wv.index2word, model.wv.syn0))
#docvecs_np_array=np.toarray(model.docvecs)
#print(docvecs_np_array)

10
timeformodel:0.0015859200002523721
timeforbuildvocab:14.178272426666808
timefortraintillnow:68.33685845333275
timefortraintillnow:138.00302037333313
timefortraintillnow:205.98682325333266
timefortraintillnow:273.8067985066664
timefortraintillnow:341.2111381333334
timefortraintillnow:408.90608170666655
timefortraintillnow:474.2509905066663
timefortraintillnow:543.2719257600002
timefortraintillnow:613.0832294399997
timefortraintillnow:680.0454545066668
20
timeformodel:0.07694378666656121
timeforbuildvocab:13.926432426666906
timefortraintillnow:66.25986730666682
timefortraintillnow:132.6492957866667
timefortraintillnow:198.9069175466666
timefortraintillnow:265.8340219733336
timefortraintillnow:332.73209472000053
timefortraintillnow:399.4692974933332
timefortraintillnow:466.0077009066672
timefortraintillnow:532.3983500800005
timefortraintillnow:598.4977856000005
timefortraintillnow:664.654827946667
30
timeformodel:0.08297429333379114
timeforbuildvocab:13.94455679999919
timefortraintilln

MemoryError: 

In [22]:
#docvecs_np_array=np.toarray(model.docvecs)
print(model.docvecs.doctag_syn0)

AttributeError: 'KMeans' object has no attribute 'docvecs'

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

X=model.docvecs.doctag_syn0
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=10, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10,
    n_clusters=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [21]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:


NameError: name 'vectorizer' is not defined

In [32]:
documents = TaggedLineDocument('comments4.txt')
model = Doc2Vec(documents, size=10, window=8, min_count=5, workers=4)


In [33]:
print(model.syn0)

AttributeError: 'Doc2Vec' object has no attribute 'syn0'

In [38]:
model.most_similar('uc')

[('again', 0.9715061783790588),
 ('add', 0.9681113958358765),
 ('least', 0.9595438838005066),
 ('21', 0.9575897455215454),
 ('650', 0.9541219472885132),
 ('450', 0.9538404941558838),
 ('fly', 0.9531024694442749),
 ('rather', 0.9528695940971375),
 ('82', 0.9520964622497559),
 ('bounce', 0.9511395692825317)]

In [53]:
model.docvecs.doctag_syn0.shape

(12007, 10)

In [54]:
import gensim
import os
import collections
import smart_open
import random

In [55]:

def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [56]:
datafile='comments4.txt'
train_corpus = list(read_corpus(datafile))

In [57]:
train_corpus[:2]

[TaggedDocument(words=['srei', 'infra', 'who', 'financed', 'bikaner', 'suratgarh', 'rd', 'and', 'partner', 'of', 'mbl', 'infra', 'is', 'rising', 'so', 'mbl', 'infra', 'will', 'also', 'rise'], tags=[0]),
 TaggedDocument(words=['srei', 'infra', 'who', 'financed', 'bikaner', 'suratgarh', 'rd', 'and', 'partner', 'of', 'mbl', 'infra', 'is', 'rising', 'so', 'mbl', 'infra', 'will', 'also', 'rise'], tags=[1])]

In [59]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)

In [60]:
model.build_vocab(train_corpus)

In [62]:
%time model.train(train_corpus, total_examples=model.corpus_count)

Wall time: 27.5 s


8357080

In [63]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [64]:
collections.Counter(ranks)

Counter({0: 2401,
         1: 325,
         2: 184,
         3: 126,
         4: 101,
         5: 78,
         6: 62,
         7: 48,
         8: 53,
         9: 47,
         10: 44,
         11: 34,
         12: 28,
         13: 25,
         14: 29,
         15: 28,
         16: 28,
         17: 24,
         18: 20,
         19: 24,
         20: 23,
         21: 22,
         22: 14,
         23: 21,
         24: 16,
         25: 14,
         26: 25,
         27: 14,
         28: 13,
         29: 12,
         30: 10,
         31: 14,
         32: 13,
         33: 6,
         34: 13,
         35: 12,
         36: 7,
         37: 18,
         38: 9,
         39: 12,
         40: 11,
         41: 12,
         42: 7,
         43: 6,
         44: 8,
         45: 12,
         46: 8,
         48: 10,
         49: 8,
         50: 10,
         51: 7,
         52: 13,
         53: 7,
         54: 13,
         55: 5,
         56: 9,
         57: 10,
         58: 9,
         59: 8,
         60: 10

In [72]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (5017): «buy with target of stop loss of»

Similar Document (4870, 0.9171402454376221): «news flash important update for all traders bsensemcx check urgently here googlzdatfl»



In [69]:
from random import shuffle
from gensim import utils
def sentences_perm(self):
    shuffle(self.sentences)
    return self.sentences

def to_array(self):
    self.sentences = []
    for source, prefix in self.sources.items():
        with utils.smart_open(source) as fin:
            for item_no, line in enumerate(fin):
                self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
    return self.sentences
